# Sets de train y test

El aprendizaje supervisado busca **aprender a partir de datos conocidos** (etapa de desarrollo) para **hacer predicciones sobre datos nuevos** (sistema en prod). Queremos:

1. Un modelo "bueno" (que hagan buenas predicciones)
2. Una **estimación confiable** de la performance del modelo

Si entrenamos un modelo con todos nuestros datos y luego evaluamos el modelo usando esos mismos datos, no podríamos **saber qué tan bien puede funcionar nuestro modelo con datos que nunca vio**.

La **partición en train y test** nos permite **evaluar si un modelo aprende patrones que generalizan** a nuevos datos. Si un modelo hace buenas predicciones sobre una observación dada, queremos que sea porque aprendió las características relevantes del proceso generador de datos, y no porque ya haya visto esa observación en particular.

En otras palabras, la partición train-test permite tener una **estimación confiable de la performance de los modelos**.

Es fundamental **_ocultar_ los datos de test** incluso de nosotros mismos. No se pueden utilizar para mejorar el modelo; solo se pueden usar para evaluar el modelo al final de todo el proceso. A la hora de **tomar decisiones de desarrollo** (selección de metodología, ajuste de hiperparámetros, selección de features, etc.) **no podemos usar el set de test**. 

Si hiciéramos esto, las nuevas versiones del modelo que vamos creando estarían indirectamente moldeadas por haber visto los datos de test. Así como un modelo corre el riesgo de sobreajustarse a los datos de entrenamiento, nosotros corremos el **riesgo de sobreajustar los modelos a los datos de test** por medio de la exploración de alternativas. La estimación de la performance en test sería optimista, y por ende, poco confiable.

Esto no significa que _siempre_ necesitemos un conjunto de test independiente; pero generalmente es así. 

Resumiendo:

![train_test](img/train_test.png)

Cuando desarrollamos modelos:

       datos           -->  métrica(datos) optimista [overfitting / modelos optimizan métrica de train]
|-------------------|
  Train   |  Val.      -->  métrica(val.) optimista  [meta-overfitting / NOSOTROS optimizamos métrica(val)] 
|-------------------|
  Train | Val. |Test   -->  métrica(test) confiable 
|--------------|----|

## Tipos de _split_

Una propiedad clave de los sets de train y test es que deben ser **representativos de los datos a los que se enfrentará el modelo** en el futuro. Si bien no siempre es posible lograr esto (¡el futuro es desconocido por definición!), podemos tomar decisiones para **simular el escenario de uso** y evitar errores.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit

Generamos datos al azar para un escenario de clasificación multi-clase:

In [15]:
n = 200

np.random.seed(42)
X = np.random.uniform(0, 1, size=(n, 5))
y = np.random.choice(["A","B","C"], size=n, p=[.1, .5, .4])

### Random split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

### Stratified split

En general es importante que todas las clases relevantes de un set de datos estén representadas en train y en test (generalmente las clases relevantes son las del _target_ pero podría tratarse de un feature categórico). 

En un dataset lo suficientemente grande, un _random split_ conservará en promedio todas las clases con sus respectivas frecuencias relativas. Pero en **datasets pequeños y/o desbalanceados**, es probable que una partición completamente aleatoria reduzca sensiblemente o elimine por completo alguna clase de alguno de los sets; la _partición estratificada_ permite que se conserven todas las clases con seguridad.

In [18]:
pd.Series(y).value_counts(normalize=True)

C    0.495
B    0.405
A    0.100
dtype: float64

In [20]:
print(
    pd.Series(y).value_counts(normalize=True),
    pd.Series(y_train).value_counts(normalize=True),
    pd.Series(y_test).value_counts(normalize=True),
    sep="\n"
)

C    0.495
B    0.405
A    0.100
dtype: float64
C    0.50625
B    0.38750
A    0.10625
dtype: float64
B    0.475
C    0.450
A    0.075
dtype: float64


In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42)

In [22]:
print(
    pd.Series(y).value_counts(normalize=True),
    pd.Series(y_train).value_counts(normalize=True),
    pd.Series(y_test).value_counts(normalize=True),
    sep="\n"
)

C    0.495
B    0.405
A    0.100
dtype: float64
C    0.49375
B    0.40625
A    0.10000
dtype: float64
C    0.5
B    0.4
A    0.1
dtype: float64


### Time-wise split

Cuando los datos están indexados en distintos momentos del tiempo, es importante hacer una partición según el tiempo. Las **observaciones de test debén ser siempre posteriores al set de train**. 

Si la partición fuera aleatoria, nuestro modelo podría ver los datos tanto antes como después de las fechas que está tratando de predecir -- entonces no sería representativo del caso de uso más típico: cuando usamos datos históricos para predecir el futuro.

In [23]:
tiempo = np.random.randint(2000, 2011, size=n) # 10 periodos

In [25]:
min_test_frac = 0.2
# min_test_size = 50 # alternativa

In [26]:
tiempo_freq_cumulative = pd.Series(tiempo).value_counts(
    normalize=True).sort_index(ascending=False).cumsum()

In [27]:
tiempo_freq_cumulative

2010    0.090
2009    0.170
2008    0.265
2007    0.355
2006    0.435
2005    0.545
2004    0.610
2003    0.705
2002    0.800
2001    0.905
2000    1.000
dtype: float64

In [28]:
# primer periodo que supera min_test_frac
tiempo_start_test = (tiempo_freq_cumulative >= min_test_frac).idxmax() 

In [29]:
is_test = tiempo >= tiempo_start_test

In [30]:
X_train = X[~is_test]
y_train = y[~is_test]
X_test = X[is_test]
y_test = y[is_test]

### Subject-wise split

Cuando contamos con **múltiples observaciones por sujeto**, la mayor parte de las veces nos interesa hacer predicciones sobre sujetos que están fuera de nuestros datos. En este caso, hay que ser muy cuidadosos en la partición de los datos. 

En particular, **no debe haber sujetos que estén en ambos sets**. 

Si hubiera sujetos de train en test, a un modelo podría resultarle fácil hacer buenas predicciones sobre test porque podría sobreajustarse a las particularidades de esos sujetos específicos en lugar de aprender los patrones relevantes para generalizar. Esto sucedería si hay **correlaciones espurias** entre los sujetos y el _target_ (es decir, correlaciones presentes en nuestra muestra que no son representativas del verdadero proceso generador).

Al igual que en la _partición temporal_, es fundamental que el split train-test replique lo más fielmente posible el escenario de uso: el funcionamiento del modelo en producción. Solo de esta manera podemos tener una estimación confiable de la verdadera capacidad predictiva de nuestro modelo. 

In [31]:
ids = np.random.choice(list("abcdefghij"), size=n) # 10 sujetos

In [33]:
pd.Series(ids).value_counts()

b    26
e    25
a    22
i    22
f    20
j    19
g    18
h    18
d    15
c    15
dtype: int64

In [34]:
gss = GroupShuffleSplit(n_splits=1, test_size=.2, random_state=42)
for train_idx, test_idx in gss.split(X, y, ids):
    X_train = X[train_idx]
    y_train = y[train_idx]
    X_test = X[test_idx]
    y_test = y[test_idx]

In [35]:
np.unique(ids[train_idx])

array(['a', 'c', 'd', 'e', 'f', 'g', 'h', 'j'], dtype='<U1')

In [36]:
np.unique(ids[test_idx])

array(['b', 'i'], dtype='<U1')

Para pensar:

* ¿Cómo partimos los datos en cada uno de los siguientes escenarios? Y ya que estamos, ¿cuáles son los features?

>1. Queremos predecir quién gana cada partido de la NBA. Tenemos los resultados de partidos de los últimos 10 años y estadísticas de cada uno.
>2. Queremos predecir cuándo un conductor está usando el celular mientras maneja. Tenemos un dataset etiquetado de imágenes de muchos conductores usando/no usando el celular.
>3. Queremos predecir si va a llover o no. Tenemos datos históricos diarios de lluvia y otros datos meteorológicos. 

* Por último, imaginemos que trabajamos en un hospital que decide tercerizar el desarrollo de un modelo de predicción de mortalidad de pacientes en una UTI a un consultor externo. El hospital nos provee un dataset con datos de pacientes que estuvieron en la UTI en los últimos 2 años. 

>¿Qué datos le compartimos al consultor?

Pregunta 2:

Random:
train: tomas1 tomas3, fv2, fv5, --> modelo puede aprender correlacion espuria
test:  tomas2 tomas10, fv1, fv4, --> metrica optimista (no confiable)

Por sujeto:
train: tomas1, tomas3, tomas2, tomas10 --> modelo puede aprender correlacion espuria
test:  fv2, fv5, fv1, fv4, ...         --> metrica confiable




## Referencias

Recomendamos leer: 

* Howard y Gugger (2020) - Deep Learning for Coders with fastai and PyTorch (_Part 1, Chapter 1, Validation Sets and Test Sets_)

-----------------------------------------------